In [ ]:
import inspectus

In [ ]:
import numpy as np

logits = np.random.rand(10, 100)
token_ids = np.array([i for i in range(10)])
tokens = ['a', 'b', 'c', 'd', 'c', 'd', 'e', 'f', 'g', 'h']

from torch.nn.functional import nll_loss

def loss_fn(logits, target):
    import torch
    logits = torch.from_numpy(logits)
    target = torch.from_numpy(target)

    return nll_loss(logits, target).item()
    

inspectus.token_viz.visalize_token_loss(logits, token_ids, tokens, loss_fn)